In [4]:
import pandas as pd
import numpy as np
data = pd.read_excel('weather.xls', usecols=range(1,6))

In [5]:
data

,outlook,temp.,hum.,windy,class
0,sunny,85,85,False,Don’t Play
1,sunny,80,90,True,Don’t Play
2,overcast,83,78,False,Play
3,rain,70,96,False,Play
4,rain,68,80,False,Play
5,rain,65,70,True,Don’t Play
6,overcast,64,65,True,Play
7,sunny,72,95,False,Don’t Play
8,sunny,69,70,False,Play
9,rain,75,80,False,Play


In [6]:
 data = data.sort_values(by=['temp.'])

In [7]:
data

,outlook,temp.,hum.,windy,class
6,overcast,64,65,True,Play
5,rain,65,70,True,Don’t Play
4,rain,68,80,False,Play
8,sunny,69,70,False,Play
3,rain,70,96,False,Play
13,rain,71,80,True,Don’t Play
7,sunny,72,95,False,Don’t Play
11,outlook,72,90,True,Play
9,rain,75,80,False,Play
10,sunny,75,70,True,Play


In [8]:
import math
from collections import Counter

In [9]:
num_attr_name = 'temp.'
class_attr_name = 'class'
discr_result = []
df_len = len(data)

In [10]:
def eval_entropy(class_set):
    values_count = []
    given_split_count = class_set.count()
    entropy = 0
    for val in class_set.value_counts():
        values_count.append(val)
        entropy += -val/given_split_count * math.log2(val/given_split_count) 
    entropy *= given_split_count / df_len
    return entropy

class_set = pd.Series(['no', 'no', 'yes'], [1,2,3])

In [11]:
class_set

1     no
2     no
3    yes
dtype: object

In [12]:
eval_entropy(class_set)

0.19677767872596202

In [13]:
split = pd.DataFrame({'temp.': [71, 72, 72], 'class': ['no', 'no', 'yes']})

In [14]:
split

,temp.,class
0,71,no
1,72,no
2,72,yes


In [34]:
discr_result = []  
def eval_entropy(num_arr, parent_size):
    class_counter = count_class(num_arr,num_val_classes)
    entropy = 0
    given_split_size = 0
    for key, val in class_counter.items():
        given_split_size += val
    #the very formula
    for key, val in class_counter.items():
        probability = val/given_split_size
        entropy += -probability * math.log2(probability)
    entropy *= given_split_size/parent_size
    return entropy
def count_class(num_arr, num_val_classes):
    class_counter = {}
    for val in num_arr:
        for class_val in num_val_classes[val]:
            if not (class_val in class_counter):
                class_counter[class_val] = 1
            else:
                class_counter[class_val] += 1
    return class_counter
def prepare_dataset_for_entrop_discr(dataset):
    num_val_classes = dict()
    num_arr = []
    for val in data["temp."]:
        num_val_classes[val] = []
    for key in num_val_classes:
        num_arr.append(key)
    for index, row in data.iterrows():
        num_val_classes[row[1]].append(row[-1])
    return num_arr, num_val_classes
# there will be storage of numeric value and its classes

def splitter(num_arr, split_point):
    first_part = []
    second_part = []
    for val in  num_arr:
        if val < float(split_point):
            first_part.append(val)
        elif val > float(split_point):
            second_part.append(val)
    return (first_part, second_part)
   
def entropy_based_discr(num_arr):
    class_counter = count_class(num_arr,num_val_classes)
    parent_size = 0
    for key, val in class_counter.items():
        parent_size += val
    
    if len(num_arr) == 1:
        discr_result.append(num_arr)
        return
    if eval_entropy(num_arr, parent_size) == 0:
        discr_result.append(num_arr)
        return
    
    min_entropy = 'unsetted'
    for index, num_val in enumerate(num_arr):
        split_point = num_val + 0.5
        if index == len(num_arr) - 1:
            break
        first_part, second_part = splitter(num_arr, split_point)
        entropy = eval_entropy(first_part, parent_size) + eval_entropy(second_part, parent_size)
        if min_entropy == 'unsetted':
            # min_entropy = (split_point, entropy)
            min_entropy = float(split_point), entropy
        elif entropy < min_entropy[1]:
            min_entropy = float(split_point), entropy
    print(min_entropy)
    first_part, second_part = splitter(num_arr, min_entropy[0])
    entropy_based_discr(first_part)
    entropy_based_discr(second_part)
    
num_arr, num_val_classes = prepare_dataset_for_entrop_discr(data)
entropy_based_discr(num_arr)

(83.5, 0.8268850944895277)
(80.5, 0.8001741038928494)
(75.5, 0.8011735447551751)
(72.5, 0.7635472023399721)
(70.5, 0.7955659970750351)
(65.5, 0.4)
(64.5, 0.0)
(71.5, 0.6666666666666666)


In [35]:
discr_result

[[64], [65], [68, 69, 70], [71], [72], [75], [80], [81, 83], [85]]